In [2]:
import os
import pandas as pd
import numpy as np 
import h5py

In [3]:
!pip install keras
!pip install keras_vggface
!pip install pandas
!pip install scikit_image
!pip install h5py
!pip install keras_applications

In [12]:
from keras.utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.optimizers import Adam

In [5]:
from tensorflow.python.lib.io import file_io
from skimage.transform import resize
from keras import backend as K

In [6]:
from keras.callbacks import CSVLogger,TensorBoard, LearningRateScheduler,Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

In [7]:
from keras import layers
from keras.layers import Activation, Convolution2D, Conv2D, Dropout, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Input, MaxPooling2D, SeparableConv2D

In [8]:
from keras.models import Model
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [9]:
img_height, img_width = 139, 139

In [10]:
num_classes         = 7     
epochs_top_layers   = 5
epochs_all_layers   = 100
batch_size          = 128

In [13]:
base_model = InceptionV3( 
    include_top = False,
    weights= 'imagenet',
    input_shape = (img_height, img_width, 3))

87916544/87910968 [==============================] - 36s 0us/step


In [14]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(num_classes, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 139, 139, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 69, 69, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 69, 69, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 69, 69, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [15]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [16]:
def get_data(dataset):
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), img_height, img_width, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  
        single_image = np.asarray(single_image).reshape(48, 48) 
        single_image = resize(single_image, (img_height, img_width), order = 3, mode = 'constant') 
        ret = np.empty((img_height, img_width, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = to_categorical(data['emotion'])

    return images, labels    

In [ ]:
train_dataset='/Users/anushkapatil/Desktop/thesis/training_dataset.csv'
eval_dataset='/Users/anushkapatil/Desktop/thesis/eval_dataset.csv'

In [2]:
train_data_x, train_data_y  = get_data(train_dataset)
val_data  = get_data(eval_dataset)

NameError: name 'get_data' is not defined

In [3]:
folder= '/Users/anushkapatil/Desktop/thesis/Inception-v3'

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)


In [ ]:
train_generator = train_datagen.flow(
    train_data_x,
    train_data_y,
    batch_size  = batch_size)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(
    optimizer   = Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0), 
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

In [ ]:
tensorboard_top_layers = TensorBoard(
    log_dir         = folder + '/logs_top_layers',
    histogram_freq  = 0,
    write_graph     = True,
    write_grads     = False,
    write_images    = True)


In [ ]:
model.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  # samples_per_epoch / batch_size
    epochs              = epochs_top_layers,                            
    validation_data     = val_data,
    callbacks           = [tensorboard_top_layers])

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
model.compile(
    optimizer   = SGD(lr = 1e-4, momentum = 0.9, decay = 0.0, nesterov = True),
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])


In [ ]:
tensorboard_all_layers = TensorBoard(
    log_dir         = folder + '/logs_all_layers',
    histogram_freq  = 0,
    write_graph     = True,
    write_grads     = False,
    write_images    = True)


In [ ]:
reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor= 0.4,
    patience= 5,
    mode = 'auto',
    min_lr= 1e-6)


In [5]:
early_stop = EarlyStopping(monitor = 'val_loss',patience = 20, mode = 'auto')

NameError: name 'EarlyStopping' is not defined

In [ ]:
class ModelCheckpoint(Callback):

    def __init__(self, filepath, folder, monitor = 'val_loss', verbose = 0, save_best_only = False, save_weights_only = False, mode = 'auto', period = 1):
        super(ModelCheckpoint, self).__init__()
        self.monitor = monitor
        self.verbose= verbose
        self.filepath = filepath
        self.folder = folder
        self.save_best_only = save_best_only
        self.save_weights_only= save_weights_only
        self.period= period
        self.epochs_since_last_save= 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, ' 'fallback to auto mode.' % (mode), RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch = epoch + 1, **logs)
            if self.save_best_only:
                current = logs.get(self.monitor)
                if current is None:
                    warnings.warn('Can save best model only with %s available, ' 'skipping.' % (self.monitor), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s improved from %0.5f to %0.5f,' ' saving model to %s' % (epoch + 1, self.monitor, self.best, current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.model.save_weights(filepath, overwrite=True)
                        else:
                            self.model.save(filepath, overwrite=True)
                            with file_io.FileIO(filepath, mode='r') as input_f:
                                with file_io.FileIO(self.folder + '/checkpoints/' + filepath, mode='w+') as output_f:
                                    output_f.write(input_f.read())
                    else:
                        if self.verbose > 0:
                            print('\nEpoch %05d: %s did not improve' %
                                  (epoch + 1, self.monitor))
            else:
                if self.verbose > 0:
                    print('\nEpoch %05d: saving model to %s' % (epoch + 1, filepath))
                if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                else:
                    self.model.save(filepath, overwrite=True)
                    with file_io.FileIO(filepath, mode='r') as input_f:
                        with file_io.FileIO(self.folder + '/checkpoints/' + filepath, mode='w+') as output_f:
                            output_f.write(input_f.read())

In [ ]:
check_point = ModelCheckpoint(
filepath= 'Inception-v3_{epoch:02d}_{val_loss:.2f}.h5',
folder = folder,
monitor = 'val_loss', 
save_best_only= True,
mode = 'auto',
period= 1)


In [ ]:
model.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  
    epochs              = epochs_all_layers,                        
    validation_data     = val_data,
    callbacks           = [tensorboard_all_layers, reduce_lr, early_stop, check_point])


In [ ]:
model.save('Inception-v3.h5')

with file_io.FileIO('Inception-v3.h5', mode='r') as input_f:
    with file_io.FileIO(folder + '/Inception-v3.h5', mode='w+') as output_f: 
        output_f.write(input_f.read())